# Efficient Initialization of Large Models

* [Blog](https://lightning.ai/pages/community/efficient-initialization-of-large-models/)

We will be using `EleutherAI/pythia-1b` for example.

In [1]:
import lightning as L
import torch
import time
from pathlib import Path

In [2]:
from lit_gpt import GPT, Tokenizer

In [3]:
checkpoint_path = "lit-gpt/checkpoints/EleutherAI/pythia-1b/lit_model.pth"
model_name = "pythia-1b"
checkpoints = torch.load(checkpoint_path)

In [ ]:
t0 = time.time()
model = GPT.from_name(model_name)
model.load_state_dict(checkpoints)
print(f"Time to load model: {time.time() - t0:.02f} seconds.")
print("This is your old nn.Module:", isinstance(model, torch.nn.Module))

In [ ]:
accelerator = "mps"

In [5]:
fabric = L.Fabric(accelerator=accelerator)
t0 = time.time()
with fabric.device:
    model = GPT.from_name(model_name)
model.load_state_dict(checkpoints)
print(f"Time to load model: {time.time() - t0:.02f} seconds.")

Time to load model: 7.44 seconds.


In [6]:
next(model.parameters()).device

device(type='mps', index=0)

In [10]:
fabric = L.Fabric(accelerator=accelerator, precision="16-true")
t0 = time.time()
with fabric.init_module():
    model = GPT.from_name(model_name)
model.load_state_dict(checkpoints)
print(f"Time to load model: {time.time() - t0:.02f} seconds.")

Time to load model: 9.66 seconds.


In [10]:
tokenizer = Tokenizer(Path("lit-gpt/checkpoints/EleutherAI/pythia-1b"))

prompt = "Hello, my name is"
encoded = tokenizer.encode(prompt)
prompt_length = encoded.size(0)